# Hogg Family Homestead

# Question 1

This is a project for comparing neighbourhoods in Toronto. First, everyone knows West is best, and if you can afford it, stay south of Eglinton. Stay the hell outta Rexdale. 

In [5]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
!conda install -c conda-forge geopy --yes #because I can't remember if I installed this already
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
!conda install -c conda-forge folium=0.5.0 --yes # still can't remember
import folium 
from bs4 import BeautifulSoup
import lxml
print('All libraries imported.')

Solving environment: done

# All requested packages already installed.



AttributeError: module 'numpy.linalg.lapack_lite' has no attribute '_ilp64'

Look! A whole bunch of error messages that make no sense! I'm going to guess that libraries already installed means I did this already, so let's run with it and move on...


In [6]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})

NameError: name 'BeautifulSoup' is not defined

Wait, you said all libraries were imported. Where's my BeautifulSoup?

In [7]:
from bs4 import BeautifulSoup

In [8]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})

In [9]:
#get headers:
headers=table.findAll('th')
for i, head in enumerate(headers): headers[i]=str(headers[i]).replace("<th>","").replace("</th>","").replace("\n","")

In [10]:
rows=table.findAll('tr')
rows=rows[1:len(rows)]

In [11]:
for i, row in enumerate(rows): rows[i] = str(rows[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","")

In [12]:
df=pd.DataFrame(rows)
df[headers] = df[0].str.split("</td>\n<td>", n = 2, expand = True) 
df.drop(columns=[0],inplace=True)

In [13]:
# skip not assigned boroughs:
df = df.drop(df[(df.Borough == "Not assigned")].index)
# give "Not assigned" Neighborhoods same name as Borough:
df.Neighbourhood.replace("Not assigned", df.Borough, inplace=True)

I feel compelled to point out at this point that East York was at one time "Canada's Only Borough" but Mike Harris ruined that. Thanks a lot, Mike!

In [15]:
# copy Borough value to Neighborhood if NaN:
df.Neighbourhood.fillna(df.Borough, inplace=True)
# drop duplicate rows:
df=df.drop_duplicates()

In [16]:
# extract titles from columns
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))


In [17]:
df.update(
    df.Borough.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))


In [19]:
# delete Toronto annotation from Neighbourhood:
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace(", Toronto",""))
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace("\(Toronto\)",""))

In [41]:
df2 = pd.DataFrame({'Postal code':df.Postal code.unique()})
df2['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postal code'] == x['Postal code']])) for i, x in df2.iterrows())
df2['Neighborhood']=pd.Series(list(set(df['Neighbourhood'].loc[df['Postal code'] == x['Postal code']])) for i, x in df2.iterrows())
df2['Neighborhood']=df2['Neighborhood'].apply(lambda x: ', '.join(x))
df2.dtypes

SyntaxError: invalid syntax (<ipython-input-41-15fdcd9762a0>, line 1)

This is driving me crazy! It seems that while the header is Postal code, that's not recognized as valid syntax. And why does the arrow point to the 'e' as being the problem. I would have thought the space, or a lack of a space, might be it. Anyway... I'll come back to this if I have to fix it.

In [46]:
!conda install -c conda-forge folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.12.0              |     pyhd8ed1ab_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

  branca             conda-forge/noarch::branca-0.4.2-pyhd8ed1ab_0
  folium             conda-forge/noarch::folium-0.12.0-pyhd8ed1ab_0



folium-0.12.0        | 64 KB     | ##################################### | 100% 
branca-0.4.2         | 26 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction:

In [49]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


In [50]:

List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [51]:
soup = BeautifulSoup(source, 'xml')

In [52]:
table=soup.find('table')

In [53]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighbourhood
column_names = ['Postalcode','Borough','Neighbourhood']
df = pd.DataFrame(columns = column_names)

In [54]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [55]:
df.head()

AttributeError: 'NoneType' object has no attribute 'items'

  Postalcode           Borough              Neighbourhood
0        M1A      Not assigned               Not assigned
1        M2A      Not assigned               Not assigned
2        M3A        North York                  Parkwoods
3        M4A        North York           Victoria Village
4        M5A  Downtown Toronto  Regent Park, Harbourfront

OK, so it looks like Postalcode all-one-word is the header here. At least I've figured that out. 

In [56]:
df=df[df['Borough']!='Not assigned']

In [58]:
df[df['Neighbourhood']=='Not assigned']=df['Borough']
df.head()

ValueError: cannot set using a list-like indexer with a different length than the value

In [60]:
temp_df=df.groupby('Postalcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

In [61]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [62]:
df_merge.drop(['Neighbourhood'],axis=1,inplace=True)

In [63]:
df_merge.drop_duplicates(inplace=True)

In [64]:
df_merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)

In [65]:
df_merge.head()

AttributeError: 'NoneType' object has no attribute 'items'

  Postalcode           Borough                                Neighbourhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [66]:
df_merge.shape

(103, 3)

OK, I think I can start the second question now. I'm still not sure about the whole Postcode/Postal Code thing.

In [67]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [68]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [69]:
soup = BeautifulSoup(source, 'xml')

In [70]:
table=soup.find('table')

In [71]:
column_names=['Postalcode','Borough','Neighbourhood']
df = pd.DataFrame(columns=column_names)

In [72]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

df.head()

AttributeError: 'NoneType' object has no attribute 'items'

  Postalcode           Borough              Neighbourhood
0        M1A      Not assigned               Not assigned
1        M2A      Not assigned               Not assigned
2        M3A        North York                  Parkwoods
3        M4A        North York           Victoria Village
4        M5A  Downtown Toronto  Regent Park, Harbourfront

In [73]:
df=df[df['Borough']!='Not assigned']

In [74]:
df[df['Neighbourhood']=='Not assigned']=df['Borough']
df.head()

ValueError: cannot set using a list-like indexer with a different length than the value

In [75]:
temp_df=df.groupby('Postalcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)


In [76]:
df_merge = pd.merge(df, temp_df, on='Postalcode')


In [77]:
df_merge.drop(['Neighbourhood'],axis=1,inplace=True)

In [78]:
df_merge.drop_duplicates(inplace=True)

In [79]:
df_merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)
df_merge.head()

AttributeError: 'NoneType' object has no attribute 'items'

  Postalcode           Borough                                Neighbourhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [81]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [80]:
df_merge.shape

(103, 3)

# Question 2

In [90]:
geo_df=pd.read_csv('https://cocl.us/Geospatial_data')

Reading the CSV doesn't seem to have worked. We can try the other way.


In [ ]:
https://cocl.us/Geospatial_data

In [91]:
geo_df.head()

AttributeError: 'NoneType' object has no attribute 'items'

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [92]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='Postalcode')

In [93]:
geo_data=geo_merged[['Postalcode','Borough','Neighbourhood','Latitude','Longitude']]

In [94]:
geo_data.head()

AttributeError: 'NoneType' object has no attribute 'items'

  Postalcode      Borough                           Neighbourhood   Latitude  \
0        M1B  Scarborough                          Malvern, Rouge  43.806686   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476  

# Question 3

I'm going to import all the libraries again, because I'm not sure they all worked

In [95]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors


print('Libraries imported.')

AttributeError: module 'numpy.linalg.lapack_lite' has no attribute '_ilp64'

Another stupid error message I can't figure out. But what the hell, it seems everything else worked despite it.

In [96]:
List_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(List_url).text

In [97]:
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')
column_names=['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns=column_names)

In [98]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
        df.head()

In [99]:
df=df[df['Borough']!='Not assigned']

In [100]:
df[df['Neighborhood']=='Not assigned']=df['Borough']

ValueError: cannot set using a list-like indexer with a different length than the value

In [101]:
df.head()

AttributeError: 'NoneType' object has no attribute 'items'

  Postalcode           Borough                                 Neighborhood
2        M3A        North York                                    Parkwoods
3        M4A        North York                             Victoria Village
4        M5A  Downtown Toronto                    Regent Park, Harbourfront
5        M6A        North York             Lawrence Manor, Lawrence Heights
6        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

Another error message, but still the final output is prety much what I expect. Meh.

In [102]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [103]:

df_merge = pd.merge(df, temp_df, on='Postalcode')

In [104]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [105]:
df_merge.drop_duplicates(inplace=True)

In [106]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [107]:
df_merge.head()

AttributeError: 'NoneType' object has no attribute 'items'

  Postalcode           Borough                                 Neighborhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [108]:
df_merge.shape

(103, 3)

In [109]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [110]:
geo_df=pd.read_csv('https://cocl.us/Geospatial_data')

In [111]:
geo_df.head()

AttributeError: 'NoneType' object has no attribute 'items'

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [112]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='Postalcode')

In [113]:
geo_merged.head()

AttributeError: 'NoneType' object has no attribute 'items'

  Postalcode   Latitude  Longitude      Borough  \
0        M1B  43.806686 -79.194353  Scarborough   
1        M1C  43.784535 -79.160497  Scarborough   
2        M1E  43.763573 -79.188711  Scarborough   
3        M1G  43.770992 -79.216917  Scarborough   
4        M1H  43.773136 -79.239476  Scarborough   

                             Neighborhood  
0                          Malvern, Rouge  
1  Rouge Hill, Port Union, Highland Creek  
2       Guildwood, Morningside, West Hill  
3                                  Woburn  
4                               Cedarbrae  

In [114]:
geo_data=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]
geo_data.head()

AttributeError: 'NoneType' object has no attribute 'items'

  Postalcode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Malvern, Rouge  43.806686   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476  

In [115]:
toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data.head()

AttributeError: 'NoneType' object has no attribute 'items'

   Postalcode          Borough                    Neighborhood   Latitude  \
37        M4E     East Toronto                     The Beaches  43.676357   
41        M4K     East Toronto    The Danforth West, Riverdale  43.679557   
42        M4L     East Toronto  India Bazaar, The Beaches West  43.668999   
43        M4M     East Toronto                 Studio District  43.659526   
44        M4N  Central Toronto                   Lawrence Park  43.728020   

    Longitude  
37 -79.293031  
41 -79.352188  
42 -79.315572  
43 -79.340923  
44 -79.388790  

In [116]:
CLIENT_ID = 'QEF4SFROUUVOQKNPAFNNRDUW4ACAWSYYG312LM3BKEDDKPIZ' # your Foursquare ID
CLIENT_SECRET = 'CCBFW1WDVTUYDLPFX3SR0QBB5R2UKRYOZ3F1JDJ3PLQMPDRM' # your Foursquare Secret
VERSION = '20180604'

In [117]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [118]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [119]:
toronto_venues.head()

AttributeError: 'NoneType' object has no attribute 'items'

  Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  The Beaches              43.676357              -79.293031   
1  The Beaches              43.676357              -79.293031   
2  The Beaches              43.676357              -79.293031   
3  The Beaches              43.676357              -79.293031   
4  The Beaches              43.676357              -79.293031   

                                Venue  Venue Latitude  Venue Longitude  \
0                   Glen Manor Ravine       43.676821       -79.293942   
1  The Big Carrot Natural Food Market       43.678879       -79.297734   
2                 Grover Pub and Grub       43.679181       -79.297215   
3                       Upper Beaches       43.680563       -79.292869   
4                 Seaspray Restaurant       43.678888       -79.298167   

      Venue Category  
0              Trail  
1  Health Food Store  
2                Pub  
3       Neighborhood  
4   Asian Restaurant  

In [120]:
toronto_venues.groupby('Neighborhood').count()

AttributeError: 'NoneType' object has no attribute 'items'

                                                    Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            58   
Brockton, Parkdale Village, Exhibition Place                           24   
Business reply mail Processing Centre, South Ce...                     17   
CN Tower, King and Spadina, Railway Lands, Harb...                     15   
Central Bay Street                                                     59   
Christie                                                               15   
Church and Wellesley                                                   75   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             32   
Davisville North                                                        8   
Dufferin, Dovercourt Village                                           15   

In [121]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_onehot.shape

(1602, 230)

In [122]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

AttributeError: 'NoneType' object has no attribute 'items'

                                        Neighborhood   Airport  \
0                                        Berczy Park  0.000000   
1       Brockton, Parkdale Village, Exhibition Place  0.000000   
2  Business reply mail Processing Centre, South C...  0.000000   
3  CN Tower, King and Spadina, Railway Lands, Har...  0.066667   
4                                 Central Bay Street  0.000000   

   Airport Food Court  Airport Lounge  Airport Service  Airport Terminal  \
0            0.000000        0.000000         0.000000          0.000000   
1            0.000000        0.000000         0.000000          0.000000   
2            0.000000        0.000000         0.000000          0.000000   
3            0.066667        0.133333         0.133333          0.133333   
4            0.000000        0.000000         0.000000          0.000000   

   American Restaurant  Antique Shop  Aquarium  Art Gallery  Art Museum  \
0                  0.0           0.0       0.0     0.017241    0.000000

In [124]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [125]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

AttributeError: 'NoneType' object has no attribute 'items'

                                        Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place                  Café   
2  Business reply mail Processing Centre, South C...           Yoga Studio   
3  CN Tower, King and Spadina, Railway Lands, Har...        Airport Lounge   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                Bakery          Cocktail Bar        Farmers Market   
1        Breakfast Spot             Nightclub           Coffee Shop   
2            Smoke Shop         Auto Workshop               Brewery   
3       Airport Service      Airport Terminal               Airport   
4        Sandwich Place                  Café    Italian Restaurant   

  5th Most Common Venue  6th Most Common Venue 7th Most Common Venue  \
0    Seafood Restaurant         

In [126]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

NameError: name 'KMeans' is not defined

In [127]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

NameError: name 'kmeans' is not defined

In [128]:
neighborhoods_venues_sorted.head()

AttributeError: 'NoneType' object has no attribute 'items'

                                        Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place                  Café   
2  Business reply mail Processing Centre, South C...           Yoga Studio   
3  CN Tower, King and Spadina, Railway Lands, Har...        Airport Lounge   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                Bakery          Cocktail Bar        Farmers Market   
1        Breakfast Spot             Nightclub           Coffee Shop   
2            Smoke Shop         Auto Workshop               Brewery   
3       Airport Service      Airport Terminal               Airport   
4        Sandwich Place                  Café    Italian Restaurant   

  5th Most Common Venue  6th Most Common Venue 7th Most Common Venue  \
0    Seafood Restaurant         

In [130]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [131]:

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

NameError: name 'toronto_merged' is not defined